In [ ]:
# This script extracts the title, link, short description, and determines number of keywords detected in each webpage

#import all libraries
import pandas as pd
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By 
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup as bs
import time, sys, requests, random

In [ ]:
# import and authorize gspread  
import gspread
from oauth2client.service_account import ServiceAccountCredentials
scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']
google_key_file = '/Users/zacharywong/Documents/ServiceAccountKey-Secret/pelagic-tracker-338302-eaf0e0e671cb.json'
credentials = ServiceAccountCredentials.from_json_keyfile_name(google_key_file, scope)
gc = gspread.authorize(credentials)

In [ ]:
# Global variables
columnName = 'Company Names (Leave Blank if Using LinkedIn Company Mining)'
nameIndex = 0;

# paths/baseurls
spreadsheet_id = '1vFXonFCyUlEKa1f0s5tvHCKeTek_sAv7rUPYfYss0Qo'
companyFilePath = '/Users/zacharywong/github/zacharywong2023/DigitalHealthWebscrape/Misc/Companies/Company Web Scrape Tool - Company Names.csv'
driver_path = '/Users/zacharywong/Documents/Work/Portfolio/DigitalHealthWebscrape/chromedriver'
pathtoFile = '/Users/zacharywong/github/zacharywong2023/DigitalHealthWebscrape/CSV Files/'
googleurl = 'https://www.google.com/'

# floats/ints
adjustDenominator = 2
maxbackOff = 120
maxResult = 1
waitTime = 5
waitRun = 0.3

# bools
useURL = False
useName = False

In [ ]:
# exponential wait backoff algorithm 
def wait():
    pass
    global waitRun
    sleepTime = waitRun + random.uniform(0, 1)
    time.sleep(sleepTime)
    waitRun = waitRun*2
    if (waitRun >= maxbackOff):
        sys.exit("error: read from sheets quota exceeded")
# Helper function: reads in values from DigitalHealthWebscrape google sheet
# Need spreadsheet ID and the cell address where the value should be read in 

In [ ]:
def readinValue(cellLocation, sheetIndex):
    sh = gc.open_by_key(spreadsheet_id)
    worksheet = sh.get_worksheet(sheetIndex)
    try:
        value = worksheet.acell(cellLocation).value
    except:
        wait()
    return value

In [ ]:
# Read Company names from csv file 
def readCompanies():
    names = []
    df = pd.read_csv(companyFilePath, usecols = [columnName])
    df[columnName]=df[columnName].fillna(' ')
    names = df[columnName].tolist()
    #print(df)
    return names 
    

In [ ]:
# Read keywords for Keyword Detection from spreadsheet
def readKeyWords():
    keywords = []
    row = 2;
    sheetIndex = 1
    isDone = False
    cellLocationColumn = 'B'
    while (isDone == False):
        cellLocationKeyWords = cellLocationColumn + str(row)
        try:
            keyword = readinValue(cellLocationKeyWords, sheetIndex)
            keywords.append(keyword)
            if(keyword == None):
                isDone = True
                break
            else:
                row += 1 
        except:
            wait()
    keywords = keywords[0:len(keywords)-1]
    print('Keywords: ', keywords)
    return keywords

In [ ]:
# Read whether to activate Keyword Detection from spreadsheet
def readCalculateLiklihood():
    #print(links, names)
    sheetIndex = 1
    cellLocationLiklihood = 'A2'
    calculateLiklihood = False
    try:
        calculateLiklihoodInput = readinValue(cellLocationLiklihood, sheetIndex)
        if (calculateLiklihoodInput == 'Yes'):
            calculateLiklihood = True
        else:
            calculateLiklihood = False
    except:
        wait()
    return calculateLiklihood

In [ ]:
# read all user inputs from spreadsheet
def readInput():
    names = readCompanies()
    calculateLiklihood = readCalculateLiklihood()
    keywords = readKeyWords()
    return names, calculateLiklihood, keywords

In [ ]:
# Extract links from 1 page of google search results
def extractLinks(soup):
    links = []
    #Tags and classes
    linksTag = 'div'
    linksClass = 'yuRUbf'
    linksAttr = 'href'
    searchLinks = soup.find_all(linksTag, class_ = linksClass)
    for h in searchLinks:
        link = h.a.get(linksAttr)
        links.append(link)
    return links 

In [ ]:
# Extract titles from 1 page of google search results
def extractTitles(soup):
    titles = [] 
    
    titlesClass = 'LC20lb MBeuO DKV0Md'
    titlesTag = 'h3'
    searchTitles = soup.find_all(titlesTag, class_= titlesClass)
    for h in searchTitles:
        titles.append(h.text)
    return titles

In [ ]:
# Extract texts from 1 page of google search results
def extractTexts(soup):
    texts = []
    textsClass = 'VwiC3b yXK7lf MUxGbd yDYNvb lyLwlc lEBKkf'
    textsTag = 'div'
    searchText = soup.find_all(textsTag, class_= textsClass)
    for h in searchText:
        fullText = h.text
        try:
            splitText = fullText.split('— ', 1)
            text = splitText[1]
            texts.append(text);
        except:
            texts.append(fullText)
    return texts;

In [ ]:
# Calculate number of keywords detected in the top result from Google 
def calculateLiklihoods(links, keywords):
    #keywords = ['personalized', 'personalization', 'machine-learning', 'AI', 'Artificial Intelligence', '24/7', 'democratizing']
    liklihoods = []
    detectedWordsAll = []
    ignore = ['[document]', 'a', 'article', 'label', 'script', 'style']
    liklihoodDenom = len(keywords) - adjustDenominator
    
    # for each url, calculate number of words detected
    for url in links: 
        detectedWords = []
        output = ''
        count = 0
        liklihood = 0
       # try: 
        res = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'})
        html_page = res.content
        soup = bs(html_page, 'html.parser')
        text = soup.find_all(text=True)
        for t in text:
            if t.parent.name not in ignore:
                output += '{} '.format(t)
        outputSub = output.split(' ')

        # check if each word matches any in keywords list 
        for word in keywords: 
            if (word in outputSub or word.capitalize() in outputSub or word + '\n' in outputSub):
                detectedWords.append(word)
                count +=1 
        liklihood = round((count / liklihoodDenom), 2)
        #print(liklihood)
        liklihoods.append(liklihood)
        detectedWordsAll.append(detectedWords)
        return liklihoods, detectedWordsAll
        #except: 
         #   print('Skipped: cannot verify SSL')
         #   detectedWordsAll.append('None')
         #   liklihoods.append('None')
         #   return liklihoods, detectedWordsAll
  

In [ ]:
      
# Search each company name on Google 
def searchGoogle(index, namesInput, googleurl, useURL, waitTime, driver):
    websiteName = namesInput[index]
    driver.get(googleurl)
    searchBar = driver.find_element(By.NAME, 'q')
    useURL = False; 
    useName = True; 
    if (useURL):
        query = "site: " + siteURL
        print('query: ' + query)
        try:
            searchBar.send_keys(query)
            searchBar.send_keys('\n')
        except Exception as e : 
            WebDriverWait(driver, waitTime).until(EC.presence_of_element_located((By.NAME, 'q')))
            searchBar.send_keys(query)
            searchBar.send_keys('\n')
    else:
        query = websiteName
        print('query: ' + query)
        searchBar.send_keys(query)
        searchBar.send_keys('\n')

In [ ]:
     
# capture links, header, and text
# pageInfo is a list of dictionaries for each page with keys/value pairs: header, link, text
# extract and load each page of results to pageInfo 
def parseHTML(driver, calculateLiklihood, keywords, maxResult, pageInfo, namesInput):
    soup = bs(driver.page_source, 'html.parser')
    links = extractLinks(soup);
    texts = extractTexts(soup);
    titles = extractTitles(soup);
    if (calculateLiklihood):
        liklihoods, detectedWordsAll = calculateLiklihoods(links[0:1], keywords)
        pageInfo = addToPageInfo(namesInput, links, texts, maxResult, pageInfo, liklihoods, detectedWordsAll)
    else:
        pageInfo = addToPageInfo(namesInput, links, texts, maxResult, pageInfo, liklihoods = None, detectedWordsAll = None)
    return pageInfo

In [ ]:
# helper function to add 1 page of results to pageInfo list
def addToPageInfo(namesInput, links, texts, maxResult, pageInfo, liklihoods, detectedWordsAll):
    index = 0;
    while (index < maxResult):
        #create new dictionary of each search results' percentage of detected words, title, link, text, and detected words
        if (liklihoods == None):
            pageInfo.append({"Name": namesInput[nameIndex], "Link": links[index], "About": texts[index]})
        else:
            detectedWords = str(detectedWordsAll[index])
            pageInfo.append({"Percentage of Keywords Detected": liklihoods[index], "Name": namesInput[nameIndex], "Link": links[index], "About": texts[index],  "Detected KeyWords": detectedWords})
        print("Percentage of Keywords Detected: ", liklihoods[index])
        print("Name: " + namesInput[nameIndex] + "Link: " + links[index] + "About: " +  texts[index] )
        index += 1
    return pageInfo

In [ ]:
# export final result to CSV 
def exportCSV(df, pathtoFile, fileName):
    # convert pageInfo to pandas dataframe and export as csv 
    df.to_csv(pathtoFile + fileName)
  

In [ ]:
  
# export final result to spreadsheet
def updateSpreadSheet(df, sheetIndex):
    sh = gc.open_by_key(spreadsheet_id)
    worksheet = sh.get_worksheet(sheetIndex)
    worksheet.clear()
    worksheet.update([df.columns.values.tolist()] + df.values.tolist())
   

In [ ]:
 
# turn dictionary of results ot pandas dataframe to prepare for export to CSV and spreadsheet
def exportDeliverables(pageInfo, pathtoFile, fileName, calculateLiklihood):
    df = pd.DataFrame(pageInfo)
    if (calculateLiklihood):
        df = df.sort_values(by = ['Percentage of Keywords Detected'], ascending=False)
        df['Percentage of Keywords Detected'] = df['Percentage of Keywords Detected'] * 100
    sheetIndex = 3
    exportCSV(df, pathtoFile, fileName)
    updateSpreadSheet(df, sheetIndex)
    return df 

In [ ]:
# run all helper functions to scrape info from Google 
def runExtraction():
    index = 0
    namesInput, calculateLiklihood, keywords = readInput()
    print("Number of Companies: " + str((len(namesInput))))
    fileName = 'CompanyWebScrape_CSV.csv'
    
    # Access chromedriver and determine path 
    service = Service(driver_path)
    driver = webdriver.Chrome(service = service)
    
    # list of dictionaries with key/value pairs: title, link, text
    # Contains all information for all search results 
    pageInfo = []
    while (index < len(namesInput)):
        try: 
            searchGoogle(index, namesInput, googleurl, useURL, waitTime, driver)
            for page in range(0, 1):
                pageInfo = parseHTML(driver, calculateLiklihood, keywords, maxResult, pageInfo, namesInput)
                index += 1
            time.sleep(waitRun)
        except: 
            print("error parsing this website")
            index += 1
        global nameIndex
        nameIndex += 1
    df = exportDeliverables(pageInfo, pathtoFile, fileName, calculateLiklihood)
    return df

In [ ]:
# Run program with output on how long program takes 
startTime = time.time()
df = runExtraction()
endTime = time.time()
timeElapsed = endTime - startTime
print('Time Elapsed: ', timeElapsed)
df